In [1]:
import random
from datetime import datetime, timedelta
import os
import json

bank_account = {}
transactions = {}
virtual_card = {}
file_name = ""
folder_path = "history\\"

# lambda functions to add or deduct from the balance
balance_add = lambda x, y: x + y
balance_deduct = lambda x, y: x - y

# to take the current date and add 4 years to it, for Card Expiration date
def date_4_years_from_today():
    today = datetime.today()
    future_date = today + timedelta(days=4 * 365)
    return future_date.strftime("%m/%y")

# checks if an account already exists
def account_does_not_exist(mode="norm"):
    if len(bank_account) == 0:
        if mode == "norm":
            print("Please create an account first.")
            return True

        if mode == "rm":
            return True


# creates an account based on user inputs and randomly generates account number
def create_account():
    global bank_account

    if bank_account != {}:
        return print("Account already created!")

    name = input("Please enter your name: ")
    phone_number = input("Please enter your phone number: ")
    birth_date = input("Please enter your birth date: ")
    email = input("Please enter your email: ")
    password = input("Please enter your password: ")
    account_number = int(random.randint(6000000000, 6999999999))

    bank_account = {
        "name": name,
        "phone number": phone_number,
        "birth date": birth_date,
        "email": email,
        "password": password,
        "account number": account_number,
        "balance": 0,
        "history": [],
    }

    create_card(name)

    transactions[account_number] = []

    print("Account created successfully! Your account number is:", account_number)
    save_data()

# to display account info
def display_account_info():
    if len(bank_account) == 0:
        return print("No account was created!")

    for info in bank_account:
        if info == "password" or info == "history":
            continue
        print(f"{info.title()}: {bank_account[info]}")

# creates the virtual card
def create_card(name):
    card_number = int(random.randint(4000000000, 4999999999))
    cvv = int(random.randint(100, 999))
    expires = date_4_years_from_today()

    global virtual_card
    virtual_card = {
        "card number": card_number,
        "card name": name,
        "cvv": cvv,
        "expires": expires,
    }

# prints out card details
def display_card_info():
    if account_does_not_exist():
        return

    for key, value in virtual_card.items():
        print(f"{key}: {value}")


# creates a json file to store account data, transactions, and card details
def save_data():
    file_suffix = str(datetime.today().strftime("%d%m%Y_%H%M%S_%f"))

    global folder_path
    global file_name
    file_name = folder_path + "bank_data_" + file_suffix + ".json"

    while True:
        try:
            with open(file_name, "w") as file:
                json.dump(
                    {
                        "account": bank_account,
                        "transactions": transactions,
                        "Card": virtual_card,
                    },
                    file,
                    indent=4,
                )
            break
        except FileNotFoundError:
            folder_name = "history"
            current_directory = os.getcwd()
            new_folder_path = os.path.join(current_directory, folder_name)
            os.makedirs(new_folder_path)

# updates the json file with new transactions
def update_file_transactions():
    global file_name

    with open(file_name, "r") as file:
        data = json.load(file)

    data["transactions"] = transactions

    with open(file_name, "w") as file:
        json.dump(data, file, indent=4)

# removes personal data after exiting or manually removing it
def remove_data():
    if account_does_not_exist("rm"):
        return

    try:
        global file_name

        with open(file_name, "r") as file:
            data = json.load(file)

        del data["account"]["phone number"]
        del data["account"]["birth date"]
        del data["account"]["email"]
        del data["account"]["password"]
        del data["account"]["balance"]
        del data["Card"]

        with open(file_name, "w") as file:
            json.dump(data, file, indent=4)

        print("Personal data deleted!")

    except FileNotFoundError:
        pass


# display balance
def display_balance():
    if account_does_not_exist():
        return

    return print(f"Your balance: {bank_account.get('balance')}")


# deposit money into the account
def deposit():
    if account_does_not_exist():
        return

    from_iban = int(input("Enter IBAN: "))
    amount = int(input("Enter deposit amount: "))

    bank_account["balance"] = balance_add(bank_account["balance"], amount)

    record_transaction(amount, from_iban, "deposit")

    print(
        f"Deposited {amount} into account {bank_account['account number']}. New balance: {bank_account['balance']}"
    )

    update_file_transactions()


# transfer money to other external accounts
def transfer():
    if account_does_not_exist():
        return

    if bank_account["balance"] <= 0:
        return print("You don't have any funds, please deposit first!")

    to_iban = int(input("Enter IBAN: "))
    transfer_amount = int(input("Enter transfer amount: "))

    if transfer_amount > bank_account["balance"]:
        return print("You can't transfer more than you have!")

    bank_account["balance"] = balance_deduct(bank_account["balance"], transfer_amount)

    record_transaction(transfer_amount, to_iban, "transfer")

    print(
        "Transfer amount: ",
        transfer_amount,
        "\nYour new balance: ",
        bank_account["balance"],
    )

    update_file_transactions()

# records transactions in a specific format
def record_transaction(amount, iban, transaction_type):
    if transaction_type == "deposit":
        transaction_date = datetime.now().strftime("%Y-%m-%d %H:%M")
        transactions[bank_account["account number"]].append(
            {"date": transaction_date, "transaction_type": "deposit", "amount": amount}
        )
        bank_account["history"].append(
            f"Deposited {amount} on {transaction_date} from IBAN: {iban}"
        )

    if transaction_type == "transfer":
        transaction_date = datetime.now().strftime("%Y-%m-%d %H:%M")
        transactions[bank_account["account number"]].append(
            {
                "date": transaction_date,
                "transaction_type": "transfer",
                "amount": amount,
            }
        )
        bank_account["history"].append(
            f"Transfer {amount} on {transaction_date} to IBAN: {iban}"
        )


# display all transactions
def show_transactions():
    if account_does_not_exist():
        return

    if bank_account["history"] == []:
        return print("No transaction recorded.")

    for transaction in bank_account["history"]:
        print(transaction)

# deletes the account from the program
def delete_account():
    if account_does_not_exist():
        return

    remove_data()

    global bank_account
    bank_account.clear()

    print("Account deleted!\n")

# main function to execute the program code
def main():
    
    # prompts users to select a service from the list
    print("\nChoose a service:\n\n1. Create an account\n2. Deposit\n3. Transfer\n4. Display Card Information\n5. Display Balance\n6. Show Transactions\n7. Display Account Info\n8. Delete account\n9. Exit App\n\n")
    prompt = int(
        input("Input: ")
    )

    # the choices users can make
    match prompt:
        case 1:
            print()
            create_account()
        case 2:
            print()
            deposit()
        case 3:
            print()
            transfer()
        case 4:
            print()
            display_card_info()
        case 5:
            print()
            display_balance()
        case 6:
            print()
            show_transactions()
        case 7:
            print()
            display_account_info()
        case 8:
            print()
            delete_account()
        case 9:
            remove_data()
            print("Exiting app...")
            exit()

# to insure the program runs until the user exits
while True:
    if __name__ == "__main__":
        main()



Choose a service:

1. Create an account
2. Deposit
3. Transfer
4. Display Card Information
5. Display Balance
6. Show Transactions
7. Display Account Info
8. Delete account
9. Exit App



Please create an account first.

Choose a service:

1. Create an account
2. Deposit
3. Transfer
4. Display Card Information
5. Display Balance
6. Show Transactions
7. Display Account Info
8. Delete account
9. Exit App



Account created successfully! Your account number is: 6511931653

Choose a service:

1. Create an account
2. Deposit
3. Transfer
4. Display Card Information
5. Display Balance
6. Show Transactions
7. Display Account Info
8. Delete account
9. Exit App



Deposited 400 into account 6511931653. New balance: 400

Choose a service:

1. Create an account
2. Deposit
3. Transfer
4. Display Card Information
5. Display Balance
6. Show Transactions
7. Display Account Info
8. Delete account
9. Exit App



Name: ِلايعممشا
Phone Number: 0558385031
Birth Date: 2001
Email: thabetb21@gmail.com
Accoun